In [1]:
import numpy as np
import pandas as pd
import scipy as scp 
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
%matplotlib inline

###### Load the user movie ratings dataset 

In [2]:
ratingDF = pd.read_csv("u.csv", names = ["user_id", "movie_id", "rating"])
ratingDF.shape

(100000, 3)

In [3]:
ratingDF.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


###### Compute Omega - dictionaries for both users and items 

In [4]:
user_dict = {}
item_dict = {}

for i, rec in ratingDF.iterrows():
    if rec['user_id'] not in user_dict:
        user_dict[rec['user_id']] = [(rec['movie_id'], rec['rating'])]
    else:
        user_dict[rec['user_id']].append((rec['movie_id'], rec['rating']))
        
    if rec['movie_id'] not in item_dict:
        item_dict[rec['movie_id']] = [(rec['user_id'], rec['rating'])]
    else:
        item_dict[rec['movie_id']].append((rec['user_id'], rec['rating']))

###### Descriptive stats...

In [5]:
''' Unique userIDs '''
print("Total unique users: ", len(ratingDF["user_id"].unique()))
print("Total unique movies: ", len(ratingDF["movie_id"].unique()))

Total unique users:  943
Total unique movies:  1682


###### MAP inference coordinate ascent algorithm 

In [6]:
l_lambda = 2
l_sigma = 0.1
d = 5

''' Initialize all vj based on gaussian distribution '''
np.random.seed(100)
vj_matrix = multivariate_normal(mean=np.zeros(d), cov=(1/l_lambda) * np.eye(d)).rvs(len(item_dict))
ui_matrix = multivariate_normal(mean=np.zeros(d), cov=(1/l_lambda) * np.eye(d)).rvs(len(user_dict))

print("vj matrix shape: ", vj_matrix.shape)
print("uj matrix shape: ", ui_matrix.shape)

vj matrix shape:  (1682, 5)
uj matrix shape:  (943, 5)


###### Helper Methods 

In [7]:
def compute_joint_likelihood(p_lambda, p_sigma):
    
    global user_dict
    global ui_matrix
    global vj_matrix
    
    loss = 0.0
    
    for user_id in user_dict:
        ui = user_id - 1
        for movie_id, mij in user_dict[user_id]:
            vj = movie_id - 1
            predicted_mij = np.dot(ui_matrix[ui], vj_matrix[vj])
            ratting_diff = (1/(2 * p_sigma * p_sigma)) * ((mij - predicted_mij) **2)
            loss += ratting_diff
    
    ''' Compute user norm & item norm'''
    total_ui_norm = 0.0
    for ui in ui_matrix:
        user_norm = (p_lambda/2) * np.linalg.norm(ui)
        total_ui_norm += user_norm
        
    total_vj_norm = 0.0
    for vj in vj_matrix:
        user_norm = (p_lambda/2) * np.linalg.norm(vj)
        total_vj_norm += user_norm
        
    return -1 * (loss + total_ui_norm + total_vj_norm)

def update_user_location(num_dimensions, p_lambda, p_sigma):
    ''' Updating user location 
    '''
    global user_dict
    global vj_matrix
    global ui_matrix
    
    for idx in range(len(user_dict)):
    
        user_id = idx + 1
        mapping_arr = np.array(user_dict[user_id])
        vj_list = mapping_arr[:, 0] - 1
        ratings = mapping_arr[:, 1]

        first_sub_part_1 = p_lambda * np.power(p_sigma, 2) * np.eye(num_dimensions)
        
        first_sub_part_2 = np.sum(np.array([np.outer(vj_matrix[vj_list][i], vj_matrix[vj_list][i]) \
                                  for i in range(vj_matrix[vj_list].shape[0])]), axis = 0)

        first_part = first_sub_part_1 + first_sub_part_2
        second_part = np.sum(ratings.reshape(-1, 1) * vj_matrix[vj_list], axis = 0)
        
        ''' update user-id mapping
            multiply first part inverse & second part
        '''
        ui = np.matmul(np.linalg.inv(first_part), second_part.reshape(-1, 1))
        ui_matrix[idx] = np.squeeze(ui)
        
def update_item_location(num_dimensions, p_lambda, p_sigma):
    ''' Updating item location 
    '''
    
    global item_dict
    global vj_matrix
    global ui_matrix
    
    for idx in range(len(item_dict)):
    
        item_id = idx + 1
        mapping_arr = np.array(item_dict[item_id])
        ui_list = mapping_arr[:, 0] - 1
        ratings = mapping_arr[:, 1]

        first_sub_part_1 = p_lambda * np.power(p_sigma, 2) * np.eye(num_dimensions)
        temp = [np.outer(ui_matrix[ui_list][i], ui_matrix[ui_list][i]) for i in range(ui_matrix[ui_list].shape[0])]
        first_sub_part_2 = np.sum(np.array(temp), axis = 0)

        first_part = first_sub_part_1 + first_sub_part_2
        second_part = np.sum(ratings.reshape(-1, 1) * ui_matrix[ui_list], axis = 0)

        ''' update item-id mapping
            multiply first part inverse & second part
        '''
        vj = np.matmul(np.linalg.inv(first_part), second_part.reshape(-1, 1))
        vj_matrix[idx] = np.squeeze(vj)

###### Main training 

In [8]:
for iteration in range(50):
    loss = compute_joint_likelihood(l_lambda, l_sigma)
    print("iteration num: {}, loss: {}".format(iteration + 1, loss))
    update_user_location(d, l_lambda, l_sigma)
    update_item_location(d, l_lambda, l_sigma)

iteration num: 1, loss: -75204183.60398236
iteration num: 2, loss: -30361025.340819918
iteration num: 3, loss: -4543629.781865825
iteration num: 4, loss: -3549122.9451645385
iteration num: 5, loss: -3336364.5658949795
iteration num: 6, loss: -3248537.181796049
iteration num: 7, loss: -3203951.4148401474
iteration num: 8, loss: -3177132.4233412785
iteration num: 9, loss: -3158407.886837847
iteration num: 10, loss: -3144992.9298362583
iteration num: 11, loss: -3134959.192335728
iteration num: 12, loss: -3126852.719301725
iteration num: 13, loss: -3120036.480098249
iteration num: 14, loss: -3114147.487439035
iteration num: 15, loss: -3108948.003045908
iteration num: 16, loss: -3104345.883391726
iteration num: 17, loss: -3100301.3996450338
iteration num: 18, loss: -3096759.358911876
iteration num: 19, loss: -3093750.1958260387
iteration num: 20, loss: -3091194.200070084
iteration num: 21, loss: -3088933.491402721
iteration num: 22, loss: -3086906.7133455756
iteration num: 23, loss: -308508

###### Compute rating and loss

In [9]:
''' Compute rating between user id 3 & movie id 302'''
np.dot(ui_matrix[2], vj_matrix[301])

4.015008758302338

In [10]:
#user_dict[3]